In [0]:
df_silver = spark.sql("select * from parquet.`abfss://silver@cardata1.dfs.core.windows.net/data`")

In [0]:
df_silver.display()

In [0]:
df_model = spark.sql("select * from cars_catalog.gold.model")
df_branch = spark.sql("select * from cars_catalog.gold.branch")
df_dealer = spark.sql("select * from cars_catalog.gold.dealer")

In [0]:
df_fact = df_silver.join(df_model, df_silver['Model_ID'] == df_model['model_id'], how='left')\
                 .join(df_branch, df_silver['Branch_ID'] == df_branch['branch_id'], how='left')\
                 .join(df_dealer, df_silver['Dealer_ID'] == df_dealer['dealer_id'], how='left')\
                .select(df_silver['revenue'], df_silver['units_sold'], df_silver['revenue_perunit'], df_model['model_key'], df_branch['branch_key'], df_dealer['dealer_key']) 

In [0]:
df_fact.display()

In [0]:
from delta.tables import DeltaTable

In [0]:
if spark.catalog.tableExists("cars_catalog.gold.factSales"):
  deltatbl = DeltaTable.forName(spark, "cars_catalog.gold.factSales")
  deltatbl.alias("t").merge(df_fact.alias("s"), "t.model_key = s.model_key AND t.branch_key = s.branch_key AND t.dealer_key = s.dealer_key")\
    .whenMatchedUpdateAll()\
        .whenNotMatchedInsertAll()\
            .execute()

else:
    df_fact.write.format("delta")\
        .mode("overwrite")\
        .option("path","abfss://gold@cardata1.dfs.core.windows.net/factSales")\
        .saveAsTable("cars_catalog.gold.factSales")

In [0]:
%sql
select * from cars_catalog.gold.factsales